In [1]:
import moldesign as mdt
from moldesign import units as u
import ipywidgets as ipy

OpenMM could not be imported; using remote docker container
PDBFixer could not be imported; using remote docker container
PySCF not installed; using remote docker container
Reading configuration from /Users/t_leeday/.moldesign/moldesign.yml


In [2]:
from moldesign.models.lammps_model import LAMMPSPotential
import parmed as med
# mdt.configure()

In [3]:
mdt.Atom


moldesign.molecules.atoms.Atom

In [4]:
mol = mdt.from_pdb('1yu8')

In [5]:
mol = mdt.Molecule(mol.residues[1:10])

In [6]:
mdmol = mdt.assign_forcefield(mol)

Connecting to docker host at unix://var/run/docker.sock... done


In [7]:
print mdmol.write('pdb')

ATOM      1  N   GLU 1   1       7.938  27.442  19.282  0.00  0.00           N  
ATOM      2  H   GLU 1   1       8.552  26.659  19.452  0.00  0.00           H  
ATOM      3  H2  GLU 1   1       8.313  28.256  19.748  0.00  0.00           H  
ATOM      4  H3  GLU 1   1       7.724  27.478  18.296  0.00  0.00           H  
ATOM      5  CA  GLU 1   1       6.670  27.173  19.940  0.00  0.00           C  
ATOM      6  HA  GLU 1   1       6.458  27.950  20.674  0.00  0.00           H  
ATOM      7  CB  GLU 1   1       5.536  27.153  18.905  0.00  0.00           C  
ATOM      8  HB2 GLU 1   1       5.761  26.361  18.191  0.00  0.00           H  
ATOM      9  HB3 GLU 1   1       4.615  26.912  19.436  0.00  0.00           H  
ATOM     10  CG  GLU 1   1       5.362  28.483  18.156  0.00  0.00           C  
ATOM     11  HG2 GLU 1   1       5.025  29.230  18.875  0.00  0.00           H  
ATOM     12  HG3 GLU 1   1       6.343  28.764  17.774  0.00  0.00           H  
ATOM     13  CD  GLU 1   1  

In [8]:
mdmol.set_energy_model(mdt.models.LAMMPSInteractive)

In [9]:
data = mdmol.energy_model.get_lammps_data()


LAMMPS output is captured by PyLammps wrapper

Welcome to amber2lammps, a program to convert Amber files to Lammps format!

Reading /tmp/tmp.crd... default_name
148
0
done.
Reading /tmp/tmp.top... default_name
148
Reading Charges...
Reading Atomic Number...
Reading Atomic Masses...
Reading Atom Types...
Reading Excluded Atoms...
Reading Non-bonded Parameter Index...
Reading Residue Labels...
Reading Residues Starting Pointers...
Reading Bond Force Constants...
Reading Equilibrium Bond Values...
Reading Angle Force Constants...
Reading Equilibrium Angle Values...
Reading Dihedral Force Constants...
Reading Dihedral Periodicity...
Reading Dihedral Phase...
Reading 1-4 Electrostatic Scaling Factor...
Reading 1-4 Van der Waals Scaling Factor...
Reading Solty...
Reading LJ A Coefficient...
Reading LJ B Coefficient...
Reading Bonds which include hydrogen...
Reading Bonds which dont include hydrogen...
Reading Angles which include hydrogen...
Reading Angles which dont include hydrogen...
Read

In [10]:
print data

LAMMPS data file for default_name

148 atoms
149 bonds
274 angles
650 dihedrals
0 impropers

11 atom types
30 bond types
67 angle types
93 dihedral types

-53.255 58.844618499999996 xlo xhi
-40.264713900000004 79.2298095 ylo yhi
-34.146 78.3984322 zlo zhi

Masses

1 14.01
2 1.008
3 12.01
4 1.008
5 1.008
6 12.01
7 16.0
8 1.008
9 16.0
10 1.008
11 1.008

Pair Coeffs

1 0.16999999991766696 3.2499985240310356
2 0.015700000004219245 1.069078461720523
3 0.10939999991572773 3.399669508450741
4 0.015700000062726104 1.9599771790073583
5 0.01570000002623629 2.649532787260222
6 0.08600000012835884 3.399669507944831
7 0.20999999984182244 2.9599219016446874
8 0.01570000009846142 2.4713530426421655
9 0.2104000002486992 3.066473387458142
10 0.0 0.0
11 0.015000000064220668 2.5996424587350853

Bond Coeffs

1 570.0 1.229
2 490.0 1.335
3 656.0 1.25
4 340.0 1.09
5 317.0 1.522
6 310.0 1.526
7 340.0 1.09
8 310.0 1.526
9 317.0 1.522
10 434.0 1.01
11 367.0 1.471
12 553.0 0.96
13 340.0 1.09
14 340.0 1.09
15 310